In [1]:
import requests
from io import StringIO
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import datetime
import time

In [2]:
def check_symbol (stock_df ,label) :
    test_dr = pd.DataFrame()
    for i in range (len(stock_df)):
        if len(stock_df.iloc[i][label]) == 4:
            test_dr = test_dr.append(stock_df[i:i+1]).reset_index(drop=True)
    return test_dr

In [3]:
def extract(Stock_list, whole_df , label):
    start = 0
    end = 0
    point = Stock_list[0]
    for i in range (len(whole_df)):
        if whole_df.iloc[i][label] >= str(point):
            #print (whole_df.iloc[i][label] , i)
            if start != 0 :
                end = i
                break
            point = Stock_list[len(Stock_list)-1]
            start = i
    test =  pd.DataFrame(whole_df[start:end+1])
    return test

In [4]:
def find_index(Stock_list, extract, label,  null) :
    institutional = pd.DataFrame()
    for i in range (len(Stock_list)):
        a = extract[extract[label] == str(Stock_list[i])].index
        if len(a)== 1  :
            institutional=institutional.append(extract.iloc[a])
        elif len(a) == 0 :
            institutional=institutional.append(null)
    return institutional.reset_index(drop=True)

In [5]:
def devide(list1 , list2) :
    out_come = list()
    for i in range (len(list1)):
        if type (list1[i]) != float and  int(list2[i].replace(",", "")) >0 :
            x = format(int(list1[i].replace(",", "")) / int(list2[i].replace(",", "")) ,'.3f')
        else :
            x = 0
        out_come.append(x)
    return out_come

In [6]:
Stock_list = pd.read_csv('name.csv')["name"]
date_list = pd.read_csv('date.csv')['date'].tolist()
whole_data = pd.DataFrame()
Stock_index = pd.read_csv('Market_index.csv')

i=0
today = datetime.date.today()
day = str (today-datetime.timedelta(days=i+553)).replace("-", "")
day

In [7]:
# generate date
i = 0
today = datetime.date.today()
day = str (today-datetime.timedelta(days=i)).replace("-", "")
record_day =  max(date_list) 
sleep_download_time = 3

while int(day) > record_day  :
#while i < 4000  :
    # get basic data
    
    r = requests.post('https://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date=' + day + '&type=ALL')
    if len(r.text) < 100 :
        print (day ," is not work day!")
        time.sleep(sleep_download_time)
        i+=1
        day = str (today-datetime.timedelta(days=i)).replace("-", "")
        continue
        
    label_ = "證券代號"
    df = pd.read_csv(StringIO(r.text.replace("=", "")),  header=["證券代號" in l for l in r.text.split("\n")].index(True)-1).drop(['Unnamed: 16'],axis = 1)
    df = df.dropna().reset_index(drop=True)
    time.sleep(sleep_download_time)
    
    # process the basic data by name list
    base_df = extract(Stock_list , df , label_).reset_index(drop=True)
    stock_df = find_index(Stock_list, base_df, label_,  df[-1:])
    
    ## get institution data
    r = requests.get('https://www.twse.com.tw/fund/T86?response=csv&date=' + day + '&selectType=ALL')
    r = r.text.replace("=", "")
    df2 = pd.read_csv(StringIO(r.replace("  \n", "")),  header=["證券代號" in l for l in r.split("\r\n")].index(True))
    df2.sort_values(by='證券代號' , ascending= True, axis =0 , inplace=True)
    df2 = df2.reset_index(drop=True).drop(['Unnamed: 19'],axis = 1)
    
    institutional = extract(Stock_list , df2 , label_).reset_index(drop=True)
    institutional = find_index(Stock_list, institutional, label_,  df2[-1:])
    time.sleep(sleep_download_time)
    ## process margin data
    r = requests.get('https://www.twse.com.tw/exchangeReport/MI_MARGN?response=csv&date=' + day + '&selectType=ALL')
    r = r.text.replace("=", "")
    df3 = pd.read_csv(StringIO(r), header=["股票代號" in l for l in r.split("\n")].index(True)).drop(['Unnamed: 16'],axis = 1)

    label_ = "股票代號"
    margin = extract(Stock_list, df3 , label_).reset_index(drop=True)
    margin = find_index(Stock_list, margin, label_,  df3[-1:])
    ratio = devide(margin['今日餘額.1'], margin['今日餘額'] )
    margin ['ratio'] = ratio 
    time.sleep(sleep_download_time)
    
    # merge to basic df
    stock_df['融資買'] = margin['買進']
    stock_df['融資賣'] = margin['賣出']
    stock_df['融資餘'] = margin['今日餘額']
    stock_df['融卷買'] = margin['買進.1']
    stock_df['融卷賣'] = margin['賣出.1']
    stock_df['融卷餘'] = margin['今日餘額.1']
    stock_df['ratio'] = margin ['ratio']

    stock_df['外資進出'] = institutional['外陸資買賣超股數(不含外資自營商)']
    stock_df['外資進'] = institutional['外陸資買進股數(不含外資自營商)']
    stock_df['外資出'] = institutional['外陸資賣出股數(不含外資自營商)']
    stock_df['投信進出'] = institutional['投信買賣超股數']
    stock_df['投信進'] = institutional['投信買進股數']
    stock_df['投信出'] = institutional['投信賣出股數']
    stock_df['自營進出'] = institutional['自營商買賣超股數(自行買賣)']
    stock_df['自營進'] = institutional['自營商買進股數(自行買賣)']
    stock_df['自營出'] = institutional['自營商賣出股數(自行買賣)']
    stock_df['避險進出'] = institutional['自營商買賣超股數(避險)']
    stock_df['避險進'] = institutional['自營商買進股數(避險)']
    stock_df['避險出'] = institutional['自營商賣出股數(避險)']
    stock_df['三大'] = institutional['三大法人買賣超股數']
    
    #stock_df  = stock_df.dropna().reset_index(drop=True)
    
    ## add date label
    date = [day for i in range(len(stock_df))]
    stock_df ["date"] = date
    whole_data = whole_data.append(stock_df)
    date_list.append(int(day))
    
    # for market index
    index_set = ['發行量加權股價指數' , '未含電子指數' , '電子類兩倍槓桿指數' ]
    r = requests.post('https://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date='+ day +'&type=IND')
    df = pd.read_csv(StringIO(r.text.replace("價格指數(臺灣證券交易所)\"\r\n", "")),  header=["指數" in l for l in r.text.split("\r\n")].index(True)).drop(['漲跌(+/-)','特殊處理註記','Unnamed: 6'],axis = 1)
    df1 = df.dropna()
    df1.columns = ['Index','收盤','漲跌','漲跌%']
    index = pd.DataFrame()
    for x in index_set :
        index = index.append(df1[df1.Index == x])
    index['date'] = [int(day) for x in range (len(index))]
    Stock_index = Stock_index.append(index)
    time.sleep(sleep_download_time)    
    print (day)
    
    i+=1
    day = str (today-datetime.timedelta(days=i)).replace("-", "")

20210330


In [8]:
date = pd.DataFrame()
date['date'] = date_list
date = date.drop_duplicates()
date = date.sort_values(by= 'date',ascending=False)
date.to_csv('date.csv', index = False)

In [9]:
date

,date
515,20210330
0,20210329
1,20210326
2,20210325
3,20210324
...,...
510,20190221
511,20190220
512,20190219
513,20190218


In [10]:
drop = whole_data.dropna().reset_index(drop=True)
whole_data = drop.drop(['漲跌(+/-)', '證券名稱' ,'最後揭示買價', '最後揭示買量', '最後揭示賣價', '最後揭示賣量'],axis = 1)

In [11]:
for l in whole_data.columns : 
    list_ = []
    for i in range(len(whole_data)) :
        k = whole_data[l][i]
        if type(k) == str :
            try :
                list_.append(float (k.replace(",", "")))
            except:
                print(l , i , k)
                list_.append(0)
                continue
        else : 
            list_.append(float(k))
    whole_data[l] = list_

In [12]:
Stock_index = Stock_index.drop_duplicates()
Stock_index = Stock_index.reset_index(drop=True)
Stock_index

,Index,收盤,漲跌,漲跌%,date
0,發行量加權股價指數,16211.7,264.85,1.66,20210303
1,未含電子指數,16115,273.49,1.73,20210303
2,電子類兩倍槓桿指數,40860.8,1287.77,3.25,20210303
3,發行量加權股價指數,15946.9,6.92,-0.04,20210302
4,未含電子指數,15841.5,60.53,-0.38,20210302
...,...,...,...,...,...
1493,未含電子指數,16878.3,141.65,0.85,20210329
1494,電子類兩倍槓桿指數,40813.5,915.94,2.3,20210329
1495,發行量加權股價指數,"16,554.90",78.93,0.48,20210330
1496,未含電子指數,"16,950.69",72.42,0.43,20210330


In [13]:
for l in ['收盤','漲跌','漲跌%'] : 
    list_ = []
    for i in range(len(Stock_index)) :
        k = Stock_index[l][i]
        if type(k) == str :
            try :
                list_.append(float (k.replace(",", "")))
            except:
                print(l , i , k)
                list_.append(0)
                continue
        else : 
            list_.append(float(k))
    Stock_index[l] = list_

In [14]:
whole_data = whole_data [whole_data['開盤價'] > 0]
whole_data

,證券代號,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,本益比,...,投信進,投信出,自營進出,自營進,自營出,避險進出,避險進,避險出,三大,date
0,1102.0,12438302.0,3427.0,581843757.0,46.85,47.00,46.60,46.80,0.00,10.96,...,31000.0,0.0,145000.0,868000.0,723000.0,0.0,0.0,0.0,-1193283.0,20210330.0
1,1103.0,11381920.0,3755.0,230867872.0,19.90,20.60,19.85,20.50,1.25,7.48,...,0.0,0.0,117000.0,117000.0,0.0,0.0,0.0,0.0,3100000.0,20210330.0
2,1104.0,1615174.0,658.0,36428096.0,22.35,22.65,22.35,22.60,0.30,11.83,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,501000.0,20210330.0
3,1108.0,1081220.0,408.0,13642613.0,12.70,12.70,12.55,12.55,0.05,12.94,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,178000.0,20210330.0
4,1109.0,1055722.0,410.0,23635766.0,22.35,22.70,22.05,22.40,0.40,7.62,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,228000.0,20210330.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
884,9944.0,128366.0,130.0,2624380.0,20.35,20.55,20.35,20.55,0.10,29.78,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19000.0,20210330.0
885,9945.0,11268114.0,6173.0,548468677.0,48.60,49.25,48.10,48.90,0.05,8.56,...,0.0,0.0,-2000.0,0.0,2000.0,66000.0,81000.0,15000.0,-225205.0,20210330.0
886,9946.0,2069923.0,1113.0,34942249.0,17.50,17.70,16.50,16.60,1.10,29.12,...,0.0,0.0,1000.0,1000.0,0.0,0.0,0.0,0.0,-366600.0,20210330.0
887,9955.0,95448.0,107.0,1749797.0,18.60,18.60,18.25,18.30,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20210330.0


In [15]:
previous = pd.read_csv('clear.csv')
previous 

,證券代號,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,本益比,...,投信進,投信出,自營進出,自營進,自營出,避險進出,避險進,避險出,三大,date
0,1102.0,8255689.0,2893.0,385583399.0,46.65,46.85,46.50,46.80,0.30,10.96,...,0.0,0.0,332000.0,376000.0,44000.0,-29000.0,0.0,29000.0,-231200.0,20210329.0
1,1103.0,2184291.0,711.0,41855616.0,19.15,19.30,19.00,19.25,0.15,7.03,...,0.0,0.0,-3000.0,0.0,3000.0,0.0,0.0,0.0,76000.0,20210329.0
2,1104.0,1143256.0,399.0,25436118.0,22.20,22.35,22.20,22.30,0.15,11.68,...,0.0,0.0,-6000.0,0.0,6000.0,0.0,0.0,0.0,254000.0,20210329.0
3,1108.0,4070726.0,1060.0,51216351.0,12.45,12.80,12.45,12.50,0.05,13.59,...,0.0,0.0,5000.0,7000.0,2000.0,0.0,0.0,0.0,962000.0,20210329.0
4,1109.0,1162126.0,462.0,25593681.0,21.80,22.25,21.70,22.00,0.35,8.24,...,0.0,0.0,-1999.0,1.0,2000.0,0.0,0.0,0.0,8001.0,20210329.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438131,9943.0,190438.0,168.0,12812017.0,67.50,67.50,67.20,67.20,0.20,11.18,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-52000.0,20190920.0
438132,9945.0,2752245.0,1096.0,116365981.0,42.60,42.60,42.10,42.10,0.50,4.38,...,0.0,81000.0,40000.0,40000.0,0.0,-56000.0,0.0,56000.0,-684265.0,20190920.0
438133,9946.0,1184358.0,469.0,27781296.0,23.50,23.60,23.30,23.60,0.10,5.58,...,0.0,0.0,-78000.0,12000.0,90000.0,0.0,0.0,0.0,107000.0,20190920.0
438134,9955.0,538156.0,334.0,10280286.0,18.80,19.20,18.80,19.15,0.35,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,44000.0,20190920.0


In [16]:
K = whole_data.append(previous).reset_index(drop=True)
K

,證券代號,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,本益比,...,投信進,投信出,自營進出,自營進,自營出,避險進出,避險進,避險出,三大,date
0,1102.0,12438302.0,3427.0,581843757.0,46.85,47.00,46.60,46.80,0.00,10.96,...,31000.0,0.0,145000.0,868000.0,723000.0,0.0,0.0,0.0,-1193283.0,20210330.0
1,1103.0,11381920.0,3755.0,230867872.0,19.90,20.60,19.85,20.50,1.25,7.48,...,0.0,0.0,117000.0,117000.0,0.0,0.0,0.0,0.0,3100000.0,20210330.0
2,1104.0,1615174.0,658.0,36428096.0,22.35,22.65,22.35,22.60,0.30,11.83,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,501000.0,20210330.0
3,1108.0,1081220.0,408.0,13642613.0,12.70,12.70,12.55,12.55,0.05,12.94,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,178000.0,20210330.0
4,1109.0,1055722.0,410.0,23635766.0,22.35,22.70,22.05,22.40,0.40,7.62,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,228000.0,20210330.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
439020,9943.0,190438.0,168.0,12812017.0,67.50,67.50,67.20,67.20,0.20,11.18,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-52000.0,20190920.0
439021,9945.0,2752245.0,1096.0,116365981.0,42.60,42.60,42.10,42.10,0.50,4.38,...,0.0,81000.0,40000.0,40000.0,0.0,-56000.0,0.0,56000.0,-684265.0,20190920.0
439022,9946.0,1184358.0,469.0,27781296.0,23.50,23.60,23.30,23.60,0.10,5.58,...,0.0,0.0,-78000.0,12000.0,90000.0,0.0,0.0,0.0,107000.0,20190920.0
439023,9955.0,538156.0,334.0,10280286.0,18.80,19.20,18.80,19.15,0.35,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,44000.0,20190920.0


In [17]:
K.columns

Index(['證券代號', '成交股數', '成交筆數', '成交金額', '開盤價', '最高價', '最低價', '收盤價', '漲跌價差',
       '本益比', '融資買', '融資賣', '融資餘', '融卷買', '融卷賣', '融卷餘', 'ratio', '外資進出', '外資進',
       '外資出', '投信進出', '投信進', '投信出', '自營進出', '自營進', '自營出', '避險進出', '避險進', '避險出',
       '三大', 'date'],
      dtype='object')

In [18]:
K.to_csv('clear.csv', index = False)
Stock_index.to_csv('Market_index.csv', index = False)